In [1]:
from PIL import Image
import io
import numpy as np
from scipy import ndimage

# Load the uploaded image
input_path = "icon.png"
img = Image.open(input_path)

# Mac app icon specifications
CANVAS_SIZE = 1024
ICON_SIZE = 832  # Approximately 81.25% of canvas (recommended active area)
MARGIN = (CANVAS_SIZE - ICON_SIZE) // 2  # 96 pixels on each side

In [4]:
# Convert to numpy array for processing
img_array = np.array(img)

# Step 1: Create a mask for near-white pixels (RGB all >= 250)
rgb = img_array[:, :, :3]
near_white_mask = np.all(rgb >= 96, axis=2)

# Step 2: Create corner seed points (all 4 corners)
h, w = near_white_mask.shape
corner_seeds = np.zeros_like(near_white_mask, dtype=bool)

# Mark corners (10x10 pixel areas in each corner)
corner_size = 10
corner_seeds[:corner_size, :corner_size] = True  # Top-left
corner_seeds[:corner_size, -corner_size:] = True  # Top-right
corner_seeds[-corner_size:, :corner_size] = True  # Bottom-left
corner_seeds[-corner_size:, -corner_size:] = True  # Bottom-right

# Step 3: Find all near-white pixels connected to corners using flood fill
# Use binary dilation to find connected components
structure = np.ones((3, 3), dtype=bool)  # 8-connectivity
corner_connected = ndimage.binary_propagation(
    corner_seeds, 
    mask=near_white_mask, 
    structure=structure
)

# Step 4: Make corner-connected near-white pixels transparent
img_array[corner_connected, 3] = 0  # Set alpha to 0

# Convert back to PIL Image
img_transparent = Image.fromarray(img_array, 'RGBA')

# Now proceed with the Mac app icon specifications
CANVAS_SIZE = 1024
ICON_SIZE = 832
MARGIN = (CANVAS_SIZE - ICON_SIZE) // 2

# Create a new transparent canvas
canvas = Image.new('RGBA', (CANVAS_SIZE, CANVAS_SIZE), (0, 0, 0, 0))

# Resize the transparent image to fit within the icon area
img_resized = img_transparent.copy()
img_resized.thumbnail((ICON_SIZE, ICON_SIZE), Image.Resampling.LANCZOS)

# Calculate position to center the image
paste_x = (CANVAS_SIZE - img_resized.width) // 2
paste_y = (CANVAS_SIZE - img_resized.height) // 2

# Paste the resized image onto the canvas
canvas.paste(img_resized, (paste_x, paste_y), img_resized)

# Save the result
output_path = "icon_resized.png"
canvas.save(output_path, 'PNG', optimize=True)